In [ ]:
''' ####label 기억용 개인간 차이 있음
# 0 : 스트레칭 1번 팔 당기기
  1 : 스트레칭 2번 옆구리
  2 : O 표시
  3 : 가만히 있기
  4 : X 표시
  5 : 팔벌려뛰기
  6 : 한 발 서기
  7 : 스쿼트
  8 : 런지
'''

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time, os

seq_length = 30

# MediaPipe hands model
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

created_time = int(time.time())
os.makedirs('dataset3', exist_ok=True)

folder_path='./resource/video/'

video_paths = []
for filename in os.listdir(folder_path):
    if filename.endswith(('.mp4')):  # 비디오 파일 포맷에 따라 확장자를 추가할 수 있음
        video_paths.append(os.path.join(folder_path, filename))


for idx, video_path in enumerate(video_paths):
    cap = cv2.VideoCapture(video_path)
    data = []
    #####
    with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            
            ret, image = cap.read()
            if not ret:
                print("카메라를 찾을 수 없습니다.")
                break
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            result = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                result.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
            if result.pose_landmarks is not None:
                
                joint = np.zeros((33, 4))
                for j, lm in enumerate(result.pose_landmarks.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                    
                # Compute an0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25    
                v1 = joint[[11,11,11,12,12,13,14,15,15,15,16,16,16,17,18,23,23,24,25,26,27,27,28,28,29,30], :3] # Parent joint
                v2 = joint[[12,13,23,14,24,15,16,17,19,21,18,20,22,19,20,24,25,26,27,28,29,31,30,32,31,32], :3] # Child joint
                v = v2 - v1 # [26, 3]
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    
                # Get angle using arcos of dot product
                angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,0,0,0,1,1,3,5,5,5, 6, 6, 6,7,7, 7,8, 8,10,10,10,11,11,15,15,16,17,18,18,19,19,20,20,21,22,22,23 ],:], 
                v[[1,2,3,4,2,5,6,7,8,9,10,11,12,8,9,13,9,13,11,12,14,12,14,16,17,18,19,20,21,22,23,21,24,24,23,25,25 ],:])) 
    
                angle = np.degrees(angle) # Convert radian to degree
    
                angle_label = np.array([angle], dtype=np.float32)
                angle_label = np.append(angle_label, idx)
    
                d = np.concatenate([joint.flatten(), angle_label])
    
                data.append(d)
    
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
            if cv2.waitKey(1) == ord('q'):
                break
        
    data = np.array(data)
    print(idx, data.shape)
    np.save(os.path.join('dataset3', f'raw_{idx}_{created_time}'), data)
    
    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])
    
    full_seq_data = np.array(full_seq_data)
    print(idx, full_seq_data.shape)
    np.save(os.path.join('dataset3', f'seq_{idx}'), full_seq_data)
    ######


cap.release()
cv2.destroyAllWindows()

2024-07-01 18:42:51.023443: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 18:42:51.023685: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 18:42:51.025783: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-01 18:42:51.053377: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 18:42:51.591433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

0 (19, 170)
0 (0,)


I0000 00:00:1719826973.347011   10020 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719826973.355505  232011 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719826973.406484  231995 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719826973.421174  231987 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


1 (8, 170)
1 (0,)


I0000 00:00:1719826973.748528   10020 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719826973.757143  232054 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719826973.811642  232042 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719826973.826243  232036 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


...

I0000 00:00:1719826976.035316   10020 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719826976.043980  232223 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719826976.100996  232205 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719826976.115523  232202 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


6 (76, 170)
6 (46, 30, 170)


I0000 00:00:1719826978.992820   10020 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719826979.002441  232284 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719826979.053709  232273 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719826979.066772  232268 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


7 (173, 170)
7 (143, 30, 170)
8 (1, 170)
8 (0,)


I0000 00:00:1719826985.404859   10020 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719826985.423878  232329 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719826985.478381  232316 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719826985.490722  232307 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
